In [3]:
from pyspark.sql import SparkSession
from pyspark import SparkContext

from pyspark.sql import functions as f
from pyspark.mllib.evaluation import BinaryClassificationMetrics, MulticlassMetrics
from pyspark.ml.feature import StandardScaler, OneHotEncoder, StringIndexer, Imputer, VectorAssembler

from pyspark.ml import Pipeline, PipelineModel
from pyspark.ml.classification import LogisticRegression

In [5]:
# Create spark context
sc = SparkContext("local", "pipelines")
# Get spark session
spark = SparkSession.builder.getOrCreate()

# Get the data and place it in a spark dataframe
data = spark.read.format("csv").option("sep", ";").option("inferSchema", "true").option("header", "true").load(
    "../../chapter1/stream-classifier/data/bank/bank.csv")

# map target to numerical category
data = data.withColumn('label', f.when((f.col("y") == "yes"), 1).otherwise(0))

RuntimeError: Java gateway process exited before sending its port number

In [ ]:
# define lsit for storage stage references
stages = []

# define the transformation stages for the categorical columns
categoricalColumns = ['job', 'marital', 'education', 'contact', 'housing', 'loan', 'default', 'day']
for categoricalCol in categoricalColumns:
    StringIndexer = StringIndexer(inputCol = categoricalCol,
                                  outputCol = categoricalCol + 'Index').setHandleInvalid('keep') # keep is for unknown categories
    # Use onehotencoder to convert cat variables into binary sparseVectors
    encoder = OneHotEncoder(inputCols=[StringIndexer.getOutputCol()], outputCols=[categoricalCol + 'classVec'])
    
    # Add stages
    stages += [StringIndexer, encoder]

In [ ]:
# define impute stage for numerical columns
numericalColumns = ['age', 'balance']
numericalColumnsImputed = [x + '_imputed' for x in numericalColumns]
imputer = Imputer(inputCols=numericalColumns, outputCols=numericalColumnsImputed)
stages += [imputer]

In [ ]:
# define numerical assembler first for scaling
numericalAssembler = VectorAssembler(inputCols=numericalColumnsImputed, outputCol='numerical_cols_imputed')
stages += [imputer]

In [ ]:
# define the standad scaler stage for the numerical columns
scaler = StandardScaler(inputCol='numerical_cols_imputed', outputCol='numerical_cols_imputed_scaled')
stages += [scaler]

In [ ]:
# Perform assembly stage to bring together features
assemblerInputs = [c+"classVec" for c in categoricalColumns]+["numerical_cols_imputed_scaled"]
# features contains everything, one hot encoded and numerical
assembler = VectorAssembler(inputCols=assemblerInputs, outputCol="features")
stages += [assembler]

In [ ]:
# define the model stage at the end of the pipeline
lr = LogisticRegression(labelCol="label", featuresCol="features", maxIter=10)
stages += [lr]

In [ ]:
# Define the entire pipeline and fit and transform on the data
# Random train test split with seed
(trainingData, testData) = data.randomSplit([0.7, 0.3], seed=100)

In [ ]:
clfPipeline = Pipeline().setStages(stages).fit(trainingData)

In [ ]:
clfPipeline.transform(testData).select("prediction")